In [1]:
import os
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


In [2]:
def preprocess_text(text):
    # Suppression des caractères spéciaux
    text = re.sub(r"[^a-zA-ZÀ-ÿ0-9\s]", "", text)
    
    # Suppression des espaces après la suppression des caractères spéciaux
    text = text.strip()
    
    # Suppression de la ponctuation
    text = re.sub(r"[^\w\s]", "", text)
    # Suppression des chiffres
    text = re.sub(r"\d", "", text)
    text = text.replace('&', '')
    # Convertir le texte en minuscules
    text = text.lower()
    # Remplacer les doubles espaces par un seul espace
    text = re.sub(r'\s+', ' ', text)
    
    return text

In [3]:
def charger_fichiers_csv(chemin):
    # Obtenir la liste des fichiers dans le chemin spécifié
    fichiers = os.listdir(chemin)
    
    dataframes = []
    
    # Parcourir chaque fichier dans la liste
    for fichier in fichiers:
        if fichier.endswith('.csv'):  # Vérifier si le fichier a l'extension .csv
            chemin_fichier = os.path.join(chemin, fichier)  # Chemin complet du fichier
            
            # Charger le fichier CSV dans un DataFrame et l'ajouter à la liste
            df = pd.read_csv(chemin_fichier)
            dataframes.append(df)

    
    return df

In [14]:
def afficher_info_dataframe(df):
    # Afficher la forme du DataFrame
    print("Shape du DataFrame :", df.shape)
    
    # Afficher le contenu du DataFrame (les 5 premières lignes)
    print("Contenu du DataFrame :")
    print(df.head())



In [4]:
def group_categories(df):
    mots_cles = ['pflege', 'makeup', 'sonne', 'körper', 'düfte', 'haare', 'gesicht', 'gesundheit','lebensmittel','tier','baby spielzeug','home lifestyle','parfum','apotheke gesundheit','haushalt','ernährung']
    
    df['category'] = df['category'].astype(str)
    #df = df.dropna(subset=['category'])
    #df = df.dropna(subset=['description'])
    
    def trouver_mot_cle(categorie, mots_cles):
        for mot_cle in mots_cles:
            if mot_cle in categorie.lower():
                return mot_cle
        return None

    df['category'] = df['category'].astype(str).apply(lambda x: trouver_mot_cle(x, mots_cles))

    df_grouped = df.groupby('category')['category'].apply(list).reset_index(name='categories_groupes')

    return df_grouped

In [5]:
def regrouper_categories(df):
    mots_cles = ['pflege', 'makeup', 'sonne', 'körper', 'düfte', 'haare', 'gesicht', 'gesundheit']

    def trouver_mot_cle(categorie, mots_cles):
        for mot_cle in mots_cles:
            if mot_cle in categorie.lower():
                return mot_cle
        return None

    df['category'] = df['category'].astype(str).apply(lambda x: trouver_mot_cle(x, mots_cles))

    df_grouped = df.groupby('category')['category'].apply(list).reset_index(name='categories_groupes')

    return df_grouped

In [6]:
# -Vectorisation des données 
#- la recherche des meilleurs paramètres à l'aide de la validation croisée
#-entraînement du modèle LinearSVC
#-la prédiction des labels sur l'ensemble de test

In [7]:
def train_linear_svc_model(df):
    # Diviser les données en ensembles d'entraînement et de test (par exemple, 80% pour l'entraînement, 20% pour le test)
    X_train, X_test, y_train, y_test = train_test_split(df['description'], df['category'], test_size=0.2, random_state=42)
    
    X_train = X_train.apply(preprocess_text)
    X_test = X_test.apply(preprocess_text)
    
    # Créer une instance de Tf-idfVectorizer ou CountVectorizer
    tfidf_vectorizer = TfidfVectorizer()
    count_vectorizer = CountVectorizer()
    
    # Appliquer la vectorisation aux données d'entraînement et de test
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)
    
    X_train_count = count_vectorizer.fit_transform(X_train)
    X_test_count = count_vectorizer.transform(X_test)
    
    # Définir une pipeline avec une vectorisation Tfidf et un classificateur LinearSVC
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', LinearSVC())
    ])
    
    # Définir les paramètres à tester pour le classificateur LinearSVC
    parameters = {
        'clf__C': [0.1, 1, 10],
        'clf__max_iter': [100, 500, 1000]
    }
    
    # Cross validation
    # Utiliser la validation croisée pour trouver les meilleurs paramètres
    grid_search = GridSearchCV(pipeline, parameters, cv=5)
    grid_search.fit(X_train, y_train)
    
    # Afficher les meilleurs paramètres et le score obtenu sur l'ensemble de validation
    print("Meilleurs paramètres : ")
    print(grid_search.best_params_)
    print("Score sur l'ensemble de validation : ")
    print(grid_search.best_score_)
    
    # Créer une instance du classificateur SVM à noyau linéaire avec les meilleurs paramètres trouvés
    best_clf = LinearSVC(C=grid_search.best_params_['clf__C'], max_iter=grid_search.best_params_['clf__max_iter'])
    
    # Entraîner le modèle sur l'ensemble d'entraînement
    best_clf.fit(X_train_tfidf, y_train)
    
    # Prédire les labels sur l'ensemble de test
    predictions_SVM = best_clf.predict(X_test_tfidf)
    
    return best_clf, X_test_tfidf, y_test

In [8]:
def evaluate_and_save_model(best_clf, X_test_tfidf, y_test, model_path):
    # Faire des prédictions sur l'ensemble de test
    y_pred = best_clf.predict(X_test_tfidf)

    # Calculer la précision
    precision = precision_score(y_test, y_pred, average='weighted')
    print("Precision:", precision)

    # Calculer le score F1
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Score F1 sur l'ensemble de test:", f1)

    # Calculer l'accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy sur l'ensemble de test:", accuracy)

    # Enregistrer le modèle
    joblib.dump(best_clf, model_path)


In [11]:
def main():
    # Chemin vers les fichiers CSV
    chemin = 'C:/Users/infoMix/Desktop/PFE/Deutch'
    
    # Charger les fichiers CSV dans un DataFrame
    df = charger_fichiers_csv(chemin)
    
    # Regrouper les catégories
    df = group_categories(df)
    print(df)
    
    # Vérifier si la colonne 'description' est toujours présente dans le DataFrame
    if 'description' not in df.columns:
        print("La colonne 'description' n'est pas présente dans le DataFrame.")
        return
    
    
    
if __name__ == '__main__':
    main()

Empty DataFrame
Columns: [category, categories_groupes]
Index: []
La colonne 'description' n'est pas présente dans le DataFrame.


In [12]:
print(df.shape)

NameError: name 'df' is not defined

In [10]:
# Entraîner le modèle LinearSVC
    best_clf, X_test_tfidf, y_test = train_linear_svc_model(df)
    
    # Évaluer le modèle et enregistrer
    model_path = 'model.pkl'
    evaluate_and_save_model(best_clf, X_test_tfidf, y_test, model_path)

IndentationError: unexpected indent (844393206.py, line 2)